In [3]:
#import all libraries needed 
import os
import pandas as pd

from api_keys import gkey
import gmaps
import gmaps.datasets
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress

# from ipywidgets.embed import embed_minimal_html


ModuleNotFoundError: No module named 'api_keys'

In [ ]:
#Get Path to get to the Datasets
firepath = os.path.join('Data Sets', 'California_Fire_Incidents.csv')
singlepath = os.path.join('Data Sets','City_zhvi_single_family.csv')

#read the files 
fire = pd.read_csv(firepath)
singlecity = pd.read_csv(singlepath)

# Cleaning Data Sets

In [ ]:
#Collecting only the columns we want from Fire dataset 
clfire = fire[['Name','AcresBurned','ArchiveYear','Counties','Started','Extinguished', 'Latitude',
       'Longitude','Fatalities','Injuries','StructuresDamaged', 'StructuresDestroyed',
       'StructuresEvacuated', 'StructuresThreatened','Location', 'CanonicalUrl']]
#sort the columns by Acres Burned
clfire.sort_values(by ='AcresBurned',ascending = False)

In [ ]:
#Graph 5 
fires_dropped = clfire.drop_duplicates(subset=['CanonicalUrl'])
graph_df = fires_dropped.groupby(["ArchiveYear"]).sum()
graph5_df = graph_df[["AcresBurned"]]

graph5_df
graph5_df.plot.bar(y = "AcresBurned", figsize = (7,5), title = "California Wildfires 2013-2019")
plt.savefig('CaliforniaAcresBurned.png')
plt.show()



In [ ]:
#selecting the columns we want from Singles homes city Zillow Data set
clsinglecity = singlecity[["RegionID","SizeRank","RegionName","RegionType","StateName","State","Metro","CountyName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30']] 

clsinglecity

In [ ]:
#Grabbing fires in Northern California only
Nfire = clfire.loc[clfire['Latitude'].gt(37)==True]
Nfire.sort_values(by ='AcresBurned',ascending = False)
Nfire = pd.DataFrame(Nfire)
Nfire['Extinguished'] = Nfire['Extinguished'].astype(str)
Nfire = pd.DataFrame(Nfire)

#Standardizing the Started Data by getting rid of the time 
for index, row in Nfire.iterrows():
    start= row[4]
    finish = row[5]
    Nfire.loc[index,'Started'] = start.split("T")[0]
    Nfire.loc[index,'Extinguished'] = finish.split('T')[0]

    
#Standardizing the Started Data  by converting the Date format to match that of the City Data Year-Month-Day(number of days in month)
for index, row in Nfire.iterrows():
    date =""
    month = row[4].split('-')[1]
    year = row[4].split('-')[0]
    if month == "01" or month == "03" or month == "05" or month == "07"or month == "08"or  month == "10" or month == "12":
        Nfire.loc[index,'Started'] = f"{year}-{month}-31"
    elif month == "04" or month == "06" or month == "09" or month == "11":
        Nfire.loc[index,'Started'] = f"{year}-{month}-30"
    elif month == "02" :
        Nfire.loc[index,'Started'] = f"{year}-{month}-28"

Nfire

In [ ]:
#using only the cities in California
SingleCCal = clsinglecity.loc[clsinglecity['StateName'] =='CA']
SingleCCal

In [ ]:
county_unique = SingleCCal.CountyName.unique()
county_unique 

In [ ]:
rural_county_list = ["Alpine County", "Amador County", "Calaveras County", "Colusa County", "Glenn County", "Humboldt County", "Inyo County", " Lake County", " Lassen County", "Mariposa County", "Mendocino County", "Modoc County", "Mono County", "Nevada County", "Plumas County","Del Norte County", "Sierra County","Siskiyou County", "Tehama County", "Trinity County", "Tuolumne County"]

rural_df = SingleCCal[SingleCCal.CountyName.isin(rural_county_list)]


rural_df2 = rural_df.drop(columns=['RegionID', 'SizeRank'])

rural_df2.mean().plot.line(figsize = (10,5), title = "Rural California - Mean Single Home Value Index")

plt.savefig('RuralCalSingleHomeValues.png')
plt.show()



In [ ]:
rural_county_list = ["Alpine County", "Amador County", "Calaveras County", "Colusa County", "Glenn County", "Humboldt County", "Inyo County", " Lake County", " Lassen County", "Mariposa County", "Mendocino County", "Modoc County", "Mono County", "Nevada County", "Plumas County","Del Norte County", "Sierra County","Siskiyou County", "Tehama County", "Trinity County", "Tuolumne County"]

metro_df = SingleCCal[~SingleCCal.CountyName.isin(rural_county_list)]


metro_df2 = metro_df.drop(columns=['RegionID', 'SizeRank'])

metro_df2.mean().plot.line(figsize = (10,5), title = "Metro California - Mean Single Home Value Index")
plt.savefig('MetroCalSingleHomeValues.png')
plt.show()



In [ ]:
#retrieving a list of the city names from the Single family owned homes
Single_City_Cal_List = SingleCCal['RegionName']
Single_City_Cal_List

In [ ]:
##HIGHLIGHT THIS -NATHAN-
#Creating lata and lng columns in the Single Family owned homes
SingleCCal['Latitude'] = ""
SingleCCal['Longitude'] =""
#creating lists to store the data of failed cities 
failedindex = []
failedcity = []
Lcounter = 0
SingleCCal = pd.DataFrame(SingleCCal)
#create a for loop to find the lat and lng of the cities 
for index, row in SingleCCal.iterrows():
    #connect to the Weather API to get latitude and longitude for the cities 
    target_city = f"{row[2]}, {row[7]}, California"
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)
    gapi = requests.get(target_url)
    gapi
    Lcounter +=1

    #create a try and fail to not break the code if the information isnt found
    try:
        #grabbing the data from the API and loading it onto the DF
        SingleCCal.loc[index,'Longitude'] = gapi.json()["results"][0]["geometry"]["location"]["lng"]
        SingleCCal.loc[index,'Latitude']  = gapi.json()["results"][0]["geometry"]["location"]["lat"]
        #print successful
        print(f'{Lcounter} Found {row[2]} in Google IPA, and Adding Latitude and Longitude')
    except:
        #print Failure
        print(f'{Lcounter} Failed at finding Latitude and Longitude for {row[2]}')
        #store city information
        failedcity.append(row[2])
        failedindex.append(index)

        
        

              

In [ ]:
#print count of cities that failed and the names
print(len(failedcity))
print((failedcity))

In [ ]:
#Show the new DF
SingleCCal

#Dropping Outliers for purposes of Graphs3,4,6
SingleCCal = SingleCCal[SingleCCal.RegionName != "Truckee"]

SingleCCal

In [ ]:
#Converting column values to numbers
SingleCCal['Latitude']  = pd.to_numeric(SingleCCal['Latitude'],errors = 'coerce')
SingleCCal['Longitude']  = pd.to_numeric(SingleCCal['Longitude'],errors = 'coerce')
#Creating a DF for just Northern California
SingleNCity  = SingleCCal.loc[SingleCCal['Latitude'].gt(37)==True]
SingleNCity

In [ ]:
#Declairing a function and setting 4 variable  Max and min on  Longitude and latitude 
def singlecity(latMin, latMax, lngMin, lngMax):  
    #Create a copy of the DF of fire and single city
    SingleFire = Nfire.copy()
    #create a box of latitude and longitude using the DFs
    SingleFire = SingleFire.loc[SingleFire['Latitude'].gt(int(latMin))==True]
    SingleFire = SingleFire.loc[SingleFire['Latitude'].gt(int(latMax))==False]

    SingleFire = SingleFire.loc[SingleFire['Longitude'].gt(int(lngMin))==False]
    SingleFire = SingleFire.loc[SingleFire['Longitude'].gt(int(lngMax))==True]
    
    
    SingleCity = SingleNCity.copy()
    SingleCity = SingleCity.loc[SingleCity['Latitude'].gt(int(latMin))==True]
    SingleCity = SingleCity.loc[SingleCity['Latitude'].gt(int(latMax))==False]

    SingleCity = SingleCity.loc[SingleCity['Longitude'].gt(int(lngMin))==False]
    SingleCity = SingleCity.loc[SingleCity['Longitude'].gt(int(lngMax))==True]
    #return the DF into a list
    return [SingleCity, SingleFire]




In [ ]:
#Create a list to hold the DF CHICO/PARADISE
TwoSingleCity_Fire = []
#choose the Lat and Long
TwoSingleCity_Fire =singlecity(39.5,40.5,-120.3,-122)
#Print number of rows
print(f'There are {TwoSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {TwoSingleCity_Fire[1]["Name"].count()} Fires')
#Choose DF to show

#cleaning the City Data
#picking the columns
TwoSingleCity_Fire[0] = TwoSingleCity_Fire[0][["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30','Latitude',"Longitude"]] 
#if they are missing any data we drop the city 
TwoSingleCity_Fire[0] = TwoSingleCity_Fire[0].dropna(how='any')


#picking the columns we wanted 
TwoSingleCity_Fire[1] = TwoSingleCity_Fire[1][['Name','AcresBurned','ArchiveYear','Started','Extinguished','Latitude',"Longitude"]]
#dropping missing data 
TwoSingleCity_Fire[1] = TwoSingleCity_Fire[1].dropna(how= 'any')
#dropping the duplicates(there were duplicates for the counties)
TwoSingleCity_Fire[1] = TwoSingleCity_Fire[1].drop_duplicates(subset = 'Name')
#organizing by achers burned
TwoSingleCity_Fire[1] = TwoSingleCity_Fire[1].sort_values(by ='AcresBurned',ascending = False)
#print number of rows after cleaning
print(f'There are {TwoSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {TwoSingleCity_Fire[1]["Name"].count()} Fires')



In [ ]:

#create new Data Frame with columns we want
fillempty = TwoSingleCity_Fire[1][['Name','Started']]
#choose the top best fires(They were already sorted by acres burned)
fillempty = fillempty.head(5)
#make sure the DF is a DF and not a list
fillempty= pd.DataFrame(fillempty)
#print Dates for Reference against decimal points
print(fillempty)
#taking the year and making it a whole number then taking the month and day and turning it into a decimal
for index, row in fillempty.iterrows():
    #create variable to hold date value
    strdate = str(row[1])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date string
    if month == 1:
        fillempty.loc[index,'Started'] = year +((day)/365)
    if month == 2:
        fillempty.loc[index,'Started'] = year +((day+28)/365)   
    if month == 3:
        fillempty.loc[index,'Started'] = year +((day+28+31)/365) 
    if month == 4:
        fillempty.loc[index,'Started'] = year +((day+28+31+30)/365) 
    if month == 5:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 

#create location Speicific DF 
Twofillempty = fillempty.copy()

#present DF
Twofillempty


In [ ]:
####Graph number 6

#Create copy of Fire DF and choose columns we want to keep
CityLine = TwoSingleCity_Fire[0][["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30']] 
#select top 10 cities
CityLine = CityLine.head(10)
#index the DF by Region name
CityLine = CityLine.set_index('RegionName')
#flip(transpose) the indexed values(RegionName) with the columns
CityLine = CityLine.T
#reset the index to allow the days to be edited
CityLine = CityLine.reset_index()
#taking the year and making it a whole number then taking the month and day and turning it into a decimal
for index, row in CityLine.iterrows():
        #create variable to hold date value
    strdate = str(row[0])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date string
    if month == 1:
        CityLine.loc[index,'index'] = year +((day)/365)
    if month == 2:
        CityLine.loc[index,'index'] = year +((day+28)/365)   
    if month == 3:
        CityLine.loc[index,'index'] = year +((day+28+31)/365) 
    if month == 4:
        CityLine.loc[index,'index'] = year +((day+28+31+30)/365) 
    if month == 5:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 
#create location Speicific DF 
Twoline = CityLine.copy()


#pull location from Speicific DF 
fillempty = Twofillempty.copy()

#Create plot
ax = Twoline.plot(x='index',figsize= (9,9))
#create color pallet
color = ['black','red']
#create color counter for 5 fires
colorcount = 0
#create line styles for fires 
linestyle = ["", "-" ,"--", '-.', ':']
#create a for loop to create different line styles for the fires
for index, row in fillempty.iterrows():
    if colorcount == 0:
        ax.axvline( row[1],color=color[1], label = row[0])
    else:
        ax.axvline( row[1],color=color[0], linestyle = linestyle[colorcount], label = row[0])
    colorcount+=1
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Chico/Paradise Area \nPrice of Single Households VS Time \nWith Major Fire Constants" )
#save file outside notebook
plt.savefig("Images/ChicoHouseWithFireConst.png")
#show Graph
plt.show()

In [ ]:
##Graph#3

#plot line data and create graph
ax = Twoline.plot(x='index',figsize= (10,10))
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Chico/Paradise Area \nPrice of Single Households VS Time" )
#Save file outside notebook
plt.savefig("Images/ChicoHouse.png")
#show graph
plt.show()

In [ ]:
#####Graph number 4
#create copy of city data
heatcity = TwoSingleCity_Fire[0]
#choose columns wanted
heatcity = heatcity[['RegionName','Latitude',"Longitude"]]
#choose top 25 cities
heatcity = heatcity.head(25)
# create lat and lng list            
locationsheatcity = heatcity[['Latitude',"Longitude"]]
#get fire data
heatfire = TwoSingleCity_Fire[1]
#choose columns wanted
heatfire = heatfire[['Name','AcresBurned','Latitude',"Longitude"]]
# choose top 25 fires
heatfire = heatfire.head(25)
#create lat and lng list of fire data
locheatfire = heatfire[['Latitude','Longitude']]
#create how intense we want the heat areas to be off acers burned
heat = heatfire['AcresBurned'].astype(float)
#create map figure
fig = gmaps.figure()
#create heat locations
heatH = gmaps.heatmap_layer(locheatfire,weights = heat,dissipating = False,
                           max_intensity = 1, point_radius = .01)
#Create info box for cities
info_box_template = """
<dl>
<dt>City</dt><dd>{RegionName}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
cityinfo = [info_box_template.format(**row) for index, row in heatcity.iterrows()]
marker = gmaps.marker_layer(locationsheatcity, info_box_content = cityinfo) 
#add city markers on map
fig.add_layer(marker)
#add heat layer on map
fig.add_layer(heatH)
#Save images outside of the notebook
embed_minimal_html('Images/chicoHeatcitiesandfire.html', views=[fig])
#show figure
fig

In [ ]:
#Create a list to hold the DF SACRAMENTO
ThreeSingleCity_Fire = []
#choose the Lat and Long
ThreeSingleCity_Fire =singlecity(38,39,-120,-122)
#print number of rows at start
print(f'There are {ThreeSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {ThreeSingleCity_Fire[1]["Name"].count()} Fires')
#Choose DF to show
ThreeSingleCity_Fire[0].head()

#cleaning the City Data
#picking the columns
ThreeSingleCity_Fire[0] = ThreeSingleCity_Fire[0][["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30','Latitude',"Longitude"]] 
#if they are missing any data we drop the city 
ThreeSingleCity_Fire[0] = ThreeSingleCity_Fire[0].dropna(how='any')


#picking the columns we wanted 
ThreeSingleCity_Fire[1] = ThreeSingleCity_Fire[1][['Name','AcresBurned','ArchiveYear','Started','Extinguished','Latitude',"Longitude"]]
#dropping missing data 
ThreeSingleCity_Fire[1] = ThreeSingleCity_Fire[1].dropna(how= 'any')
#dropping the duplicates(there were duplicates for the counties)
ThreeSingleCity_Fire[1] = ThreeSingleCity_Fire[1].drop_duplicates(subset = 'Name')
# #organizing by achers burned
ThreeSingleCity_Fire[1] = ThreeSingleCity_Fire[1].sort_values(by ='AcresBurned',ascending = False)

#print number of rows after cleaning
print(f'There are {ThreeSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {ThreeSingleCity_Fire[1]["Name"].count()} Fires')

In [ ]:

#create new Data Frame with columns we want
fillempty = ThreeSingleCity_Fire[1][['Name','Started']]
#choose the top best fires(They were already sorted by acres burned)
fillempty = fillempty.head(5)
#make sure the DF is a DF and not a list
fillempty= pd.DataFrame(fillempty)

#print Dates for Reference against decimal points
print(fillempty)

#taking the year and making it a whole number then taking the month and day and turning it into a decimal
for index, row in fillempty.iterrows():
    #create variable to hold date value
    strdate = str(row[1])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date string  
    if month == 1:
        fillempty.loc[index,'Started'] = year +((day)/365)
    if month == 2:
        fillempty.loc[index,'Started'] = year +((day+28)/365)   
    if month == 3:
        fillempty.loc[index,'Started'] = year +((day+28+31)/365) 
    if month == 4:
        fillempty.loc[index,'Started'] = year +((day+28+31+30)/365) 
    if month == 5:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 
#create location Speicific DF 
Threefillempty = fillempty.copy()

#present DF
Threefillempty


In [ ]:
####Graph number 6

#drop duplicates and create copy
CityLine = ThreeSingleCity_Fire[0].drop(ThreeSingleCity_Fire[0].loc[ThreeSingleCity_Fire[0]['RegionName']=='Pittsburg'].index)
#Choose columns we want to keep
CityLine = CityLine[["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30']] 
#select top 10 cities
CityLine = CityLine.head(10)
#index the DF by Region name
CityLine = CityLine.set_index('RegionName')
#flip(transpose) the indexed values(RegionName) with the columns
CityLine = CityLine.T
#reset the index to allow the days to be edited
CityLine = CityLine.reset_index()
#create Decimal point out of month and ay
for index, row in CityLine.iterrows():
    #create variable to hold date value
    strdate = str(row[0])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date strin    
    if month == 1:
        CityLine.loc[index,'index'] = year +((day)/365)
    if month == 2:
        CityLine.loc[index,'index'] = year +((day+28)/365)   
    if month == 3:
        CityLine.loc[index,'index'] = year +((day+28+31)/365) 
    if month == 4:
        CityLine.loc[index,'index'] = year +((day+28+31+30)/365) 
    if month == 5:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 
#create location Speicific DF 
Threeline = CityLine.copy()

#pull location from specific DF 
fillempty = Threefillempty.copy()

#Create plot
ax = Threeline.plot(x='index',figsize= (10,10))
#create color pallet
color = ['black','red']
#create color counter for 5 fires
colorcount = 0
#create line styles for fires 
linestyle = ["", "-" ,"--", '-.', ':']
#create a for loop to create different line styles for the fires
for index, row in fillempty.iterrows():
    if colorcount == 0:
        ax.axvline( row[1],color=color[1], label = row[0])
    else:
        ax.axvline( row[1],color=color[0], linestyle = linestyle[colorcount], label = row[0])
    colorcount+=1
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Sacramento/Delta Area \nPrice of Single Households VS Time \nWith Major Fire Constants" )
#save file outside notebook
plt.savefig("SacramentoHouseWithFireConst.png")
#show graph
plt.show()

In [ ]:
##Graph#3
#plot line data and create graph
ax = Threeline.plot(x='index',figsize= (7,7))
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Sacramento/ Delta \nPrice of Single Households VS Time" )
#Save file outside notebook
plt.savefig("SacramentoHouse.png")
#show graph
plt.show()

In [ ]:
#####Graph number 4
#create copy of city data
heatcity = ThreeSingleCity_Fire[0]
#choose columns wanted
heatcity = heatcity[['RegionName','Latitude',"Longitude"]]
#choose top 25 cities
heatcity = heatcity.head(25)
# create lat and lng list                        
locationsheatcity = heatcity[['Latitude',"Longitude"]]
#get fire data
heatfire = ThreeSingleCity_Fire[1]
#choose columns wanted
heatfire = heatfire[['Name','AcresBurned','Latitude',"Longitude"]]
# choose top 25 fires
heatfire = heatfire.head(25)
#create lat and lng list of fire data
locheatfire = heatfire[['Latitude','Longitude']]
#create how intense we want the heat areas to be off acers burned
heat = heatfire['AcresBurned'].astype(float)
#create map figure
fig = gmaps.figure()
#create heat locations
heatH = gmaps.heatmap_layer(locheatfire,weights = heat,dissipating = False,
                           max_intensity = 1, point_radius = .01)

#Create info box for cities
info_box_template = """
<dl>
<dt>City</dt><dd>{RegionName}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
cityinfo = [info_box_template.format(**row) for index, row in heatcity.iterrows()]
marker = gmaps.marker_layer(locationsheatcity, info_box_content = cityinfo) 
#add city markers on map
fig.add_layer(marker)
#add heat layer on map
fig.add_layer(heatH)
#Save image outside of Notebook
embed_minimal_html('SacHeatcitiesandfire.html', views=[fig])
#Show figure
fig

In [ ]:
#Create a list to hold the DF REDDING
FourSingleCity_Fire = []
#choose the Lat and Long
FourSingleCity_Fire =singlecity(40,41,-121,-123)
#pring number of rows at start

print(f'There are {FourSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {FourSingleCity_Fire[1]["Name"].count()} Fires')
#Choose DF to show
FourSingleCity_Fire[1]


#cleaning the City Data
#picking the columns
FourSingleCity_Fire[0] = FourSingleCity_Fire[0][["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30','Latitude',"Longitude"]] 
#if they are missing any data we drop the city 
FourSingleCity_Fire[0] = FourSingleCity_Fire[0].dropna(how='any')


#picking the columns we wanted 
FourSingleCity_Fire[1] = FourSingleCity_Fire[1][['Name','AcresBurned','ArchiveYear','Started','Extinguished','Latitude',"Longitude"]]
#dropping missing data 
FourSingleCity_Fire[1] = FourSingleCity_Fire[1].dropna(how= 'any')
#dropping the duplicates(there were duplicates for the counties)
FourSingleCity_Fire[1] = FourSingleCity_Fire[1].drop_duplicates(subset = 'Name')
#organizing by achers burned
FourSingleCity_Fire[1] = FourSingleCity_Fire[1].sort_values(by ='AcresBurned',ascending = False)
#print number of rows after cleanup
print(f'There are {FourSingleCity_Fire[0]["RegionName"].count()} Cities')
print(f'There are {FourSingleCity_Fire[1]["Name"].count()} Fires')


In [ ]:


#create new Data Frame with columns we want
fillempty = FourSingleCity_Fire[1][['Name','Started']]
#choose the top best fires(They were already sorted by acres burned)
fillempty = fillempty.head(5)
#make sure the DF is a DF and not a list
fillempty= pd.DataFrame(fillempty)
#print Dates for Reference against decimal points
print(fillempty)
#taking the year and making it a whole number then taking the month and day and turning it into a decimal
for index, row in fillempty.iterrows():
    #create variable to hold date value
    strdate = str(row[1])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date string
    if month == 1:
        fillempty.loc[index,'Started'] = year +((day)/365)
    if month == 2:
        fillempty.loc[index,'Started'] = year +((day+28)/365)   
    if month == 3:
        fillempty.loc[index,'Started'] = year +((day+28+31)/365) 
    if month == 4:
        fillempty.loc[index,'Started'] = year +((day+28+31+30)/365) 
    if month == 5:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        fillempty.loc[index,'Started'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 

        
#create location Speicific DF 
Fourfillempty = fillempty.copy()

#present DF
Fourfillempty


In [ ]:
####Graph number 6

#Drop outliers and create copy of DF
CityLine =FourSingleCity_Fire[0].drop(FourSingleCity_Fire[0].loc[FourSingleCity_Fire[0]['RegionName']=='Westwood'].index)
#Choose columns we want to keep
CityLine = FourSingleCity_Fire[0][["RegionName",'2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30']] 
#select top 10 cities
CityLine = CityLine.head(10)
#index the DF by Region name
CityLine = CityLine.set_index('RegionName')
#flip(transpose) the indexed values(RegionName) with the columns
CityLine = CityLine.T
#reset the index to allow the days to be edited
CityLine = CityLine.reset_index()
#taking the year and making it a whole number then taking the month and day and turning it into a decimal
for index, row in CityLine.iterrows():
    #create variable to hold date value
    strdate = str(row[0])
    #split the Date into value, Day, month, and Year
    day = strdate.split('-')[2]
    month = strdate.split('-')[1]
    year = strdate.split('-')[0]
    #turn 3 date values to integers
    day = int(day)
    month = int(month)
    year = int(year)
    #turn Month Values into number of days add it to the "Day" variable and divide by number of days
    #then add back to the DF and replace the "Started" Date string
    if month == 1:
        CityLine.loc[index,'index'] = year +((day)/365)
    if month == 2:
        CityLine.loc[index,'index'] = year +((day+28)/365)   
    if month == 3:
        CityLine.loc[index,'index'] = year +((day+28+31)/365) 
    if month == 4:
        CityLine.loc[index,'index'] = year +((day+28+31+30)/365) 
    if month == 5:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31)/365) 
    if month == 6:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30)/365) 
    if month == 7:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31)/365) 
    if month == 8:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31)/365) 
    if month == 9:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30)/365) 
    if month == 10:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31)/365)         
    if month == 11:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30)/365) 
    if month == 12:
        CityLine.loc[index,'index'] = year +((day+28+31+30+31+30+31+31+30+31+30+31)/365) 
#create location Speicific DF 
Fourline = CityLine.copy()


#pull location from specific DF 

fillempty = Fourfillempty.copy()
#Create plot
ax = Fourline.plot(x='index',figsize= (10,10))
#create color pallet
color = ['black','red']
#create color counter for 5 fires
colorcount = 0
#create line styles for fires 
linestyle = ["", "-" ,"--", '-.', ':']
#create a for loop to create different line styles for the fires
for index, row in fillempty.iterrows():
    if colorcount == 0:
        ax.axvline( row[1],color=color[1], label = row[0])
    else:
        ax.axvline( row[1],color=color[0], linestyle = linestyle[colorcount], label = row[0])
    colorcount+=1
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Redding/Shasta Area \nPrice of Single Households VS Time \nWith Major Fire Constants" )
#save file outside notebook
plt.savefig("ReddingHouseWithFireConst.png")
#show Graph
plt.show()

In [ ]:
##Graph#3
#plot line data and create graph
ax = Fourline.plot(x='index',figsize= (10,10))
#set labels
ax.set(xlabel = "Years",ylabel='Price of Single Household',title ="Redding/Shasta Area \nPrice of Single Households VS Time" )
#Save file outside notebook
plt.savefig("ReddingHouse.png")
#show graph
plt.show()

In [ ]:
#####Graph number 4
#create copy of city data
heatcity = FourSingleCity_Fire[0]
#choose columns wanted
heatcity = heatcity[['RegionName','Latitude',"Longitude"]]

#choose top 25 cities
heatcity = heatcity.head(25)
                    
# create lat and lng list            
locationsheatcity = heatcity[['Latitude',"Longitude"]]

#get fire data
heatfire = FourSingleCity_Fire[1]
#choose columns wanted
heatfire = heatfire[['Name','AcresBurned','Latitude',"Longitude"]]
# choose top 25 cities
heatfire = heatfire.head(25)
#create lat and lng list of fire data
locheatfire = heatfire[['Latitude','Longitude']]
#create how intense we want the heat areas to be off acers burned
heat = heatfire['AcresBurned'].astype(float)
#create map figure
fig = gmaps.figure()
#create heat locations
heatH = gmaps.heatmap_layer(locheatfire,weights = heat,dissipating = False,
                           max_intensity = 1, point_radius = .01)

#Create info box for cities
info_box_template = """
<dl>
<dt>City</dt><dd>{RegionName}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
cityinfo = [info_box_template.format(**row) for index, row in heatcity.iterrows()]
marker = gmaps.marker_layer(locationsheatcity, info_box_content = cityinfo) 
#add city markers on map
fig.add_layer(marker)
#add heat layer on map
fig.add_layer(heatH)
#Save image outside of Notebook
embed_minimal_html('SacHeatcitiesandfire.html', views=[fig])
#show figure
fig